## Obsługa LIDAR'a

---
W ty tutorialu pokażę jak uruchomić lidar przy pomocy biblioteki rplidar.  
na początek trzeba zainstalować bibliotekę przy pomocy komendy ```pip install rplidar-roboticia```   
można to zrobić w terminalu lub blokiem poniżej usuwając znak #

In [ ]:
#!pip install rplidar-roboticia

teraz należy zlokalizować na jakim porcie znajduje się lidar, jeżeli używasz Jetson Nano AI Kit   
to będzie to ```/dev/ttyACM1``` sprawdź czy biblioteka poprawnie wykrywa lidar uruchamiając poniższy blok  
  
**Ważne - upewnij się że lidar jest włączony, w innym wypadku nie zostanie wykryty!**

In [ ]:
%matplotlib inline

from rplidar import RPLidar
PORT_NAME = '/dev/ttyACM1'

lidar = RPLidar(PORT_NAME)
info = lidar.get_info()
if type(info) == type('string'):
    raise Exception("Buffer jest pełen, spróbuj inny port")
health = lidar.get_health()
for i in info:
    print(i,"-", info[i])
print(f'health - {health[0]}')

lidar.stop()
lidar.stop_motor()

    

---
Lidar powinien się zatrzymać a blok powinien zwrócić podobny wynik:   
```
model - 24
firmware - (1, 29)
hardware - 7
serialnumber - BC91ED93C0EA98C7A0E69BF52C4F4560
health - Good
```

jeżeli w wyniku uruchomienia instrukcji pojawia się błąd:
```
FileNotFoundError: [Errno 2] No such file or directory: '/dev/ttyUSB1'
```
spróbuj inny port

---

Teraz przetestujemy czy lidar poprawnie zbiera pomiary odpal kolejny blok!

In [ ]:
lidar.start_motor()

for i, scan in enumerate(lidar.iter_scans()):
    print('%d: Got %d measurments' % (i, len(scan)))
    if i > 8:
        break

lidar.stop_motor()

---
Lidar powinien się zakręcić a funkcja powinna zwrócić podobny wynik:
```
0: Got 127 measurments
1: Got 171 measurments
2: Got 173 measurments
3: Got 167 measurments
4: Got 172 measurments
5: Got 174 measurments
6: Got 175 measurments
7: Got 167 measurments
8: Got 172 measurments
9: Got 168 measurments
```
oznacza to że wszytko działa poprawnie!

---
Teraz przejdźmy do tworzenia prostej mapy otoczenia

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
from IPython.display import display, clear_output

def run_lidar():
    lidar = RPLidar(PORT_NAME)
    
    # wyczyszczenie danych pozostałych po poprzednim skanie
    lidar.clean_input()
    
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.set_rmax(4000)
    ax.grid(True)
    
    try:
        print("Starting to collect data...")
        
        while True:
            start_time = time.time()
            scans = []
            
            lidar.start_motor()
            for scan in lidar.iter_scans(max_buf_meas=5000):
                scans.append(scan)
                if time.time() - start_time > 2:  # Collect data for 2 seconds
                    break

            # zatrzymanie lasera na czas drukowania wykresu
            lidar.stop()
            
            # wyczyść zawartość wykresu
            ax.clear()
            ax.set_rmax(4000)
            ax.grid(True)
            
            # nanieś dane na wykres
            for scan in scans:
                angles = [item[1] for item in scan]
                distances = [item[2] for item in scan]
                ax.scatter(np.radians(angles), distances, c='r', s=5)
            
            clear_output()
            display(fig) # wyświetlenie outputu
            
            
    except Exception as e:
        print(f"An error occurred: {e}")
    except KeyboardInterrupt:
        print("Stopping...")
        lidar.stop()
        lidar.stop_motor()

if __name__ == '__main__':
    run_lidar()